In [1]:
!pip install openai langchain tiktoken pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstall

In [2]:
import os

# 환경변수 세팅하기
os.environ['OPENAI_API_KEY'] = '3a9d04966a3c43ca92de7be4b803d6f7'
os.environ['AZURE_OPENAI_ENDPOINT'] = 'https://skyfly-001.openai.azure.com/'
os.environ['OPENAI_API_TYPE'] = 'azure'
os.environ['OPENAI_API_VERSION'] = '2023-05-15'

In [6]:
from langchain.embeddings import AzureOpenAIEmbeddings

embeddings_model = AzureOpenAIEmbeddings(
    model = 'dev-text-embedding-ada-002'
)

임베딩을 거치고 나오면 좌표계로 변환된다.\
좌표계의 길이가 1536이 되는 것이다.\
따라서 어떤 텍스트를 넣어도 1536차원의 벡터로 변환된다.

In [7]:
embeddings = embeddings_model.embed_documents({
    '안녕하세요',   #0
    '제 이름은 오리알입니다.',  #1
    '이름이 무엇인가요?',   #2
    '랭체인은 유용합니다.',  #3
    'Hello World'  #4
})

len(embeddings), len(embeddings[0])

(5, 1536)

In [8]:
embedded_query_q = embeddings_model.embed_query("이 대화에 언급된 이름은 무엇입니까?")
embedded_query_a = embeddings_model.embed_query("이 대화에 언급된 이름은 오리알입니다.")
print(len(embedded_query_q), len(embedded_query_a))

1536 1536


In [ ]:
embedded_query_q

In [10]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cos_sim(A, B):
  return dot(A, B) / (norm(A) * norm(B))

In [12]:
print(cos_sim(embedded_query_q, embedded_query_a))
print(cos_sim(embedded_query_q, embeddings[1]))
print(cos_sim(embedded_query_q, embeddings[2]))

0.8924888971016415
0.9216955277777744
0.7748425804148096


## Huggingface embedding

In [14]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.0 MB/s eta 0:00:00


In [15]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = 'BAAI/bge-small-en'
model_kwargs = {'device' : 'cpu'}
encode_kwargs = {'normalize_embeddings': True}

hf = HuggingFaceBgeEmbeddings(
    model_name = model_name,
    model_kwargs = model_kwargs,
    encode_kwargs = encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
embeddings = hf.embed_documents([
    'today is Monday',
    'weather is nice today.',
    "What's the problem.",
    'Hello World',
    'I am a duck.'
])

In [19]:
BGE_query_q = hf.embed_query('Hello? Who is this?')
BGE_query_a = hf.embed_query('Hi this is egg.')

print(BGE_query_q, BGE_query_a)
print(cos_sim(BGE_query_q, BGE_query_a))
print(cos_sim(BGE_query_q, embeddings[1]))
print(cos_sim(BGE_query_q, embeddings[3]))

[-0.05165727064013481, -0.014959926716983318, 0.01763145998120308, -0.041672006249427795, 0.03451971709728241, -0.035746969282627106, 0.08076844364404678, 0.02722741849720478, 0.020027972757816315, 0.01066796388477087, 0.03253635764122009, -0.06471753865480423, -0.00818810798227787, 0.03445412963628769, 0.00769772520288825, -0.029890552163124084, 0.05117839574813843, -0.01660156063735485, -0.010365480557084084, 0.013720575720071793, 0.008365683257579803, 0.030398203060030937, -0.05815192312002182, -0.04904188960790634, 0.025917086750268936, -0.01076211966574192, -0.00444230018183589, 0.0050201998092234135, 0.006352831143885851, -0.1459188014268875, -0.024220220744609833, -0.022140031680464745, 0.013377162627875805, -0.001317657413892448, 0.005801043473184109, -0.027996554970741272, 0.007844215258955956, -0.0014156581601127982, -0.040774960070848465, -0.011248917318880558, 0.010379710234701633, -0.007679908536374569, 0.018879888579249382, -0.013210948556661606, 0.007122267037630081, -0.